In [1]:
import boto3
import sagemaker
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

session = sagemaker.Session()

predictor = sagemaker.Predictor(
    endpoint_name="sagemaker-xgboost-2025-09-27-12-03-25-568", 
    sagemaker_session=session
)

# Set serializers again (these don’t persist across notebooks)
predictor.serializer = CSVSerializer()
predictor.deserializer = JSONDeserializer()


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
# Example: single row (sepal length, sepal width, petal length, petal width)
row = [[5.1, 3.5, 1.4, 0.2]]

payload = ",".join(map(str, row[0]))
result = predictor.predict(payload)

print(result)
# → {'predictions': [{'score': [0.98, 0.01, 0.01]}]}


{'predictions': [{'score': [0.9903059005737305, 0.008187990635633469, 0.001506078871898353]}]}


In [3]:
import boto3

# Get your region
session = boto3.Session()
region = session.region_name
print(f"AWS Region: {region}")

# Get your account ID (optional, for verification)
sts = boto3.client('sts')
account_id = sts.get_caller_identity()['Account']
print(f"Account ID: {account_id}")

# Your exact endpoint URL for Postman:
endpoint_url = f"https://runtime.sagemaker.{region}.amazonaws.com/endpoints/sagemaker-xgboost-2025-09-27-12-03-25-568/invocations"
print(f"\nPostman URL: {endpoint_url}")

# Check if endpoint is active
sagemaker_client = boto3.client('sagemaker')
try:
    response = sagemaker_client.describe_endpoint(
        EndpointName='sagemaker-xgboost-2025-09-27-12-03-25-568'
    )
    print(f"Endpoint Status: {response['EndpointStatus']}")
except Exception as e:
    print(f"Error checking endpoint: {e}")

AWS Region: ap-southeast-1
Account ID: 215470142970

Postman URL: https://runtime.sagemaker.ap-southeast-1.amazonaws.com/endpoints/sagemaker-xgboost-2025-09-27-12-03-25-568/invocations
Endpoint Status: InService


In [4]:
# Test the exact same format you'll use in Postman
import boto3

runtime = boto3.client('sagemaker-runtime')

response = runtime.invoke_endpoint(
    EndpointName='sagemaker-xgboost-2025-09-27-12-03-25-568',
    ContentType='text/csv',
    Body='5.1,3.5,1.4,0.2'
)

result = response['Body'].read().decode()
print("Raw response:", result)

Raw response: 0.9903059005737305,0.008187990635633469,0.001506078871898353

